# Shuttle Data Warehouse Loader

Files expected:

* drivers.csv
* routes.csv
* vehicles.csv
* trips.csv

In [3]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# ---- 1. Connection setup ----
host_name = "localhost"       
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "Peanut168996!"          # change as needed
db_name = "shuttle_dw"

engine_str = f"mysql+pymysql://{user_id}:{pwd}@{host_ip}:{port}/{db_name}"
engine = create_engine(engine_str)

print("SQLAlchemy engine created successfully.")

SQLAlchemy engine created successfully.


In [6]:
# ---- 2. Load CSVs ----
# CSV file paths
drivers_csv = "drivers.csv"
routes_csv = "routes.csv"
vehicles_csv = "vehicles.csv"
trips_csv = "trips.csv"

# Load using pandas
drivers_df = pd.read_csv(drivers_csv)
routes_df = pd.read_csv(routes_csv)
vehicles_df = pd.read_csv(vehicles_csv)
trips_df = pd.read_csv(trips_csv)

print("CSV files loaded into pandas DataFrames.")

CSV files loaded into pandas DataFrames.


In [8]:
# ---- 3. Insert data into tables ----

# ---- Load helper using SQLAlchemy ----
from sqlalchemy import text

def load_table(df, table_name, engine):
    """Load a pandas DataFrame into a MySQL table using SQLAlchemy."""
    cols = ",".join(df.columns)
    placeholders = ",".join([f":{col}" for col in df.columns])  # named params for SQLAlchemy
    sql = text(f"REPLACE INTO {table_name} ({cols}) VALUES ({placeholders})")
    
    with engine.begin() as conn:  # auto-commit transaction
        conn.execute(sql, df.to_dict(orient="records"))
    print(f"Loaded {len(df)} rows into {table_name}")

# Load each table
load_table(drivers_df, "dim_driver", engine)
load_table(routes_df, "dim_route", engine)
load_table(vehicles_df, "dim_vehicle", engine)
load_table(trips_df, "fact_passenger_counts", engine)

Loaded 10 rows into dim_driver
Loaded 4 rows into dim_route
Loaded 10 rows into dim_vehicle
Loaded 10 rows into fact_passenger_counts


In [10]:
# ---- 4. Verification ----
with engine.connect() as conn:
    driver_count = conn.execute(text("SELECT COUNT(*) FROM dim_driver")).scalar()
    trip_count = conn.execute(text("SELECT COUNT(*) FROM fact_passenger_counts")).scalar()

print(f"dim_driver rows: {driver_count}")
print(f"fact_passenger_counts rows: {trip_count}")
print("Data verification complete.")

dim_driver rows: 10
fact_passenger_counts rows: 10
Data verification complete.


In [11]:
def preview_table(table_name, engine, limit=5):
    """Fetch and display the first few rows of a table."""
    query = text(f"SELECT * FROM {table_name} LIMIT {limit}")
    df = pd.read_sql(query, con=engine)
    print(f"\nPreview of {table_name} (showing up to {limit} rows):")
    display(df)

# Preview all tables
preview_table("dim_driver", engine)
preview_table("dim_route", engine)
preview_table("dim_vehicle", engine)
preview_table("fact_passenger_counts", engine)



Preview of dim_driver (showing up to 5 rows):


,driver_id,first_name,last_name,cdl_flag,hire_month,hire_year
0,1,Alice,Brown,Y,5,2020
1,2,Mark,Jones,N,8,2022
2,3,Rita,Singh,Y,1,2019
3,4,Tom,Williams,N,9,2021
4,5,Lily,Chen,Y,3,2018



Preview of dim_route (showing up to 5 rows):


,route_id,route_name,primary_stops,route_duration_min
0,1,Green Loop,CHP,30
1,2,Orange Loop,LIB,30
2,3,Gold Line,"BAR,HER",20
3,4,Silver Line,"JPJ,PIN",15



Preview of dim_vehicle (showing up to 5 rows):


,vehicle_id,vehicle_number,vehicle_type,electric_flag,capacity
0,101,1001,bus,N,40
1,102,E201,ejest,Y,16
2,103,V12,van,N,10
3,104,C4,car,Y,4
4,105,1002,bus,Y,40



Preview of fact_passenger_counts (showing up to 5 rows):


,trip_id,date_key,route_id,driver_id,vehicle_id,passengers,trip_duration_min,route_distance_miles
0,1001,20251001,1,1,101,32,30,5.4
1,1002,20251001,2,2,102,18,30,6.1
2,1003,20251002,3,3,103,28,20,7.2
3,1004,20251002,4,4,104,4,15,2.3
4,1005,20251003,1,5,105,36,30,5.4
